In [1]:
!pip install transformers datasets
!pip install requirements.txt
!pip install gradio
!pip install opencv-python
import cv2
!pip install tifffile matplotlib 
!pip install rasterio
import rasterio
import matplotlib.pyplot as plt
!pip install torch --upgrade
!pip install onnxscript

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.16.4 requires markupsafe>=2.0, but you have markupsafe 1.1.0 which is incompatible.
notebook 7.2.2 requires jupyterlab<4.3,>=4.2.0, but you have jupyterlab 4.3.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement requirements.txt (from versions: none)
HINT: You are attempting to install a package literally name

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Intrinsic parameters for the Sony FCB-7520 camera
K1 = np.array([[350, 0, 320],
               [0, 350, 240],
               [0, 0, 1]], dtype=np.float64)

K2 = np.array([[350, 0, 320],
               [0, 350, 240],
               [0, 0, 1]], dtype=np.float64)

dist1 = np.array([0.01, -0.02, 0.0, 0.0, 0.0], dtype=np.float64)
dist2 = np.array([0.01, -0.02, 0.0, 0.0, 0.0], dtype=np.float64)

# Load stereo images (left and right images)
img_left = cv2.imread('examples/ImageLeft/0100_L.png', 0)
img_right = cv2.imread('examples/ImageRight/0100_R.png', 0)

if img_left is None or img_right is None:
    print("Error: One or both of the images were not loaded correctly.")
else:
    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Detect ORB keypoints and compute descriptors for both images
    kp1, des1 = orb.detectAndCompute(img_left, None)
    kp2, des2 = orb.detectAndCompute(img_right, None)

    # Use BFMatcher to match the descriptors
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)

    # Sort the matches based on their distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Draw matches
    img_matches = cv2.drawMatches(img_left, kp1, img_right, kp2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imshow('Matches', img_matches)
    cv2.waitKey(0)

    # Extract the matched keypoints
    pts_left = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts_right = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Generate synthetic object points (if actual 3D points are not available)
    objp = np.zeros((len(pts_left), 3), np.float32)
    objp[:, 0] = np.arange(len(pts_left))

    # Arrays to store object points and image points
    obj_points = [objp]
    img_points_left = [pts_left]
    img_points_right = [pts_right]

    # Stereo calibration to find the rotation and translation between the two cameras
    ret, K1, dist1, K2, dist2, R, T, E, F = cv2.stereoCalibrate(
        obj_points,
        img_points_left,
        img_points_right,
        K1, dist1, K2, dist2,
        imageSize=(img_left.shape[1], img_left.shape[0]),
        R=R, T=T, E=E, F=F,
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6),
        flags=cv2.CALIB_FIX_INTRINSIC
    )

    # Print the results for debugging
    print("Rotation matrix (R):\n", R)
    print("Translation vector (T):\n", T)
    print("Essential matrix (E):\n", E)
    print("Fundamental matrix (F):\n", F)
    print("Calibration return value:", ret)

    # Calculate the baseline as the norm of the translation vector T
    if np.any(T):
        baseline = np.linalg.norm(T)
        print(f"Baseline: {baseline} meters")
    else:
        print("Translation vector (T) is zero or invalid.")


NameError: name 'R' is not defined

In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import gradio as gr
from app import inference
from app import load_model
from app import convert_to_onnx

import cv2



def get_disp(image1, image2, result_pth):
    model = load_model()
    result = inference(image1, image2, model, task='stereo')
    #result.save(result_pth)
    result = np.array(result)
    plt.axis('off')
    plt.imshow(result)
    plt.colorbar()
    plt.show()
    return result
def get_depth(focal_length,baseline,cx1,cx2,disparity):
    depth = (focal_length * baseline) / (np.abs(-disparity + (cx2 - cx1)))
    return depth


img1 = Image.open('examples/ImageLeft/0100_L.png')
img2 = Image.open('examples/ImageRight/0100_R.png')
# Convert the PIL image to a NumPy array
img_np = np.array(img1)
image_height, image_width = img_np.shape[:2]

cx = image_width / 2
cy = image_height / 2
img1 = np.array(img1)
img2 = np.array(img2)
# Initiate ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
img3 = cv2.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=0)
plt.imshow(img3), plt.show()

# FLANN parameters
FLANN_INDEX_LSH = 6
index_params = dict(algorithm = FLANN_INDEX_LSH,
                    table_number = 6,  # 12
                    key_size = 12,     # 20
                    multi_probe_level = 1)  #2

search_params = dict(checks=50)  # or pass an empty dictionary

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Match descriptors using KNN
matches = flann.knnMatch(des1, des2, k=2)
# Sort them in the order of their distance

# Store only good matches using the ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.7  * n.distance:
        good_matches.append(m)




pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
fx,fy=35,35
# Assuming you have the camera intrinsic parameters K
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

if len(pts1) >= 5 and len(pts2) >= 5:
    # Compute essential matrix
    E, mask = cv2.findEssentialMat(pts1, pts2, K)
    if E is not None:
        print("Essential Matrix:\n", E)

        # Recover pose (rotation and translation)
        _, R, t, mask_pose = cv2.recoverPose(E, pts1, pts2, K)

        # Baseline length is the magnitude of the translation vector t (initially in relative scale)
        baseline = np.linalg.norm(t)
        print("Initial Baseline (Relative): ", baseline)

        # Here you would scale the baseline based on a known distance in the scene.
        # Assuming D is the known distance in meters between two triangulated points:
        D = 5.0  # For example, 5 meters

        # Perform triangulation to estimate the 3D positions of points in the scene
        pts1_normalized = cv2.undistortPoints(pts1, K, None)
        pts2_normalized = cv2.undistortPoints(pts2, K, None)
        
        points_4d_hom = cv2.triangulatePoints(np.dot(K, np.hstack((np.eye(3), np.zeros((3, 1))))),
                                              np.dot(K, np.hstack((R, t))),
                                              pts1_normalized, pts2_normalized)

        points_4d = points_4d_hom[:3] / points_4d_hom[3]

        # Select two points and calculate the distance between them in the 3D space
        point1 = points_4d[:, 0]
        point2 = points_4d[:, 1]
        distance_3d = np.linalg.norm(point1 - point2)

        # Scale the baseline based on the known distance
        scale_factor = D / distance_3d
        scaled_baseline = baseline * scale_factor

        print("Scaled Baseline: ", scaled_baseline)
    else:
        print("Essential matrix calculation failed.")
else:
    print("Not enough points to compute the essential matrix.")

# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points (0,0,0), (1,0,0), ..., (7,5,0)
objp = np.zeros((6*9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []  # 3d point in real world space
imgpoints_left = []  # 2d points in image plane of left camera
imgpoints_right = []  # 2d points in image plane of right camera








import cv2
import numpy as np

# Function to detect and compute features using ORB
def detect_and_compute_features(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

# Function to match features using FLANN-based matcher
def match_features(descriptors1, descriptors2):
    # FLANN parameters
    index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50)  # or pass empty dictionary
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    
    # Ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    return good_matches


images_left = cv2.imread('examples/ImageLeft/0100_L.png')
images_right = cv2.imread('examples/ImageRight/0100_R.png')
    
# Convert images to grayscale
gray_left = cv2.cvtColor(images_left, cv2.COLOR_BGR2GRAY)
gray_right = cv2.cvtColor(images_right, cv2.COLOR_BGR2GRAY)
#images_left = np.array(images_left)
#images_right = np.array(images_right)
#images_left = cv2.medianBlur(images_left, 3)
#images_right = cv2.medianBlur(images_right, 3)


# Detect and compute features
keypointsL, descriptorsL = detect_and_compute_features(gray_left)
keypointsR, descriptorsR = detect_and_compute_features(gray_right)
img3 = cv2.drawKeypoints(images_left, keypointsL, None, color=(0,255,0), flags=0)
plt.imshow(img3), plt.show()

# Match features
matches = match_features(descriptorsL, descriptorsR)

# Extract matched points
matched_points_left = np.float32([keypointsL[m.queryIdx].pt for m in matches])
matched_points_right = np.float32([keypointsR[m.trainIdx].pt for m in matches])
    # Assume camera intrinsic matrix K (for both cameras, same intrinsic parameters)
fx,fy=35,35
K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0, 0, 1]])
    
    # Compute the essential matrix
E, _ = cv2.findEssentialMat(matched_points_left, matched_points_right, K)
    
    # Recover pose from essential matrix
_, R, t, _ = cv2.recoverPose(E, matched_points_left, matched_points_right, K)
    
print(f"Number of matched points: {len(matched_points_left)}")

# Check if we have enough points
if len(matched_points_left) < 6:
    print("Not enough points for calibration. Please obtain more points.")
else:
    # Proceed with stereo calibration
    # Convert matched points to the format required for calibration
    objp = np.zeros((len(matched_points_left), 3), np.float32)  # Assuming a flat plane
    objp[:, :2] = np.mgrid[0:len(matched_points_left), 0:1].T.reshape(-1, 2)
        # Append object points and image points
    obj_points = []
    img_points_left = []
    img_points_right = []

    obj_points.append(objp)
    img_points_left.append(matched_points_left)
    img_points_right.append(matched_points_right)
    # Calibrate the stereo cameras
    # Calibrate the stereo cameras using the collected points
    ret, K1, dist1, K2, dist2, R, T, E, F = cv2.stereoCalibrate(
        obj_points, img_points_left, img_points_right,
        K, None, K, None,
        gray_left.shape[::-1],  # Image size (width, height)
        flags=cv2.CALIB_FIX_INTRINSIC
    )
    # Calculate the baseline
    baseline = np.linalg.norm(T)
    print(f"The baseline (distance between cameras) is: {baseline} units")

'''
image1 = Image.open('examples/ImageLeft/292879.jpg')
image2 = Image.open('examples/ImageRight/292880.jpg')
plt.imshow(image1, cmap='plasma')
plt.imshow(image2, cmap='plasma')

plt.colorbar()
plt.show()
image1 = np.array(image1)
image2 = np.array(image2)
result1_path = 'output/result1.png'


disp_one = get_disp(image1, image2, result1_path)

#baseline = (800000 * disp_one) / 43
depth_one = get_depth(43, 26000 , 10.3,10.3, disp_one)  #focal_length, baseline, cx2, cx1, disparity

print(baseline)




import cv2
import numpy as np








image1 = Image.open('examples/ImageLeft/292879.jpg')
image2 = Image.open('examples/ImageRight/292880.jpg')
image1 = np.array(image1)
image2 = np.array(image2)
image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)


baseline = (800000 * disp_one) / 43
print(baseline)
'''


def color_normalization(image):
    # Convert the image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab_image[:, :, 0] = clahe.apply(lab_image[:, :, 0])
    # Convert back to BGR color space
    normalized_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    return normalized_image
#image1 = cv2.medianBlur(image1, 5)  # 5 is an odd number
#image2 = cv2.medianBlur(image2, 5)  # 5 is an odd number
#image1 = color_normalization(image1)
#image2 = color_normalization(image2)

#plt.imshow(image1, cmap='plasma')
#plt.imshow(image2, cmap='plasma')

#plt.colorbar()
#plt.show()
'''
image1 = np.array(image1)
image2 = np.array(image2)
image1 = cv2.resize(image1, (920, 512))
image2 = cv2.resize(image2, (920, 512))


result1_path = 'output/result1.png'
disp_one = get_disp(image1, image2, result1_path)
depth_one = get_depth(43, 82000 , 10.3,10.3, disp_one)  #focal_length, baseline, cx2, cx1, disparity


image3 = Image.open('examples/ImageLeft/0100_L.png')
image4 = Image.open('examples/ImageRight/0100_R.png')
result2_path = 'output/result2.png'
disp_two = get_disp(image3, image4, result2_path)
depth_two= get_depth(350, 15000 , 10.3,10.3, disp_two)

image4 = Image.open('examples/ImageLeft/0040_L.png')
image5 = Image.open('examples/ImageRight/0040_R.png')
result3_path = 'output/result3.png'
disp_three = get_disp(image4, image5, result3_path)
depth_three = get_depth(350, 15000 , 10.3,10.3,disp_three)


image6 = Image.open('examples/ImageLeft/0120_L.png')
image7 = Image.open('examples/ImageRight/0120_R.png')
result4_path = 'output/result4.png'
disp_four = get_disp(image6, image7, result4_path)
depth_four = get_depth(350, 15000 , 10.3,10.3,disp_four)
'''

ModuleNotFoundError: No module named 'click.shell_completion'

In [ ]:


image1 = Image.open('examples/ImageLeft/04_05_0011_l.png')
image2 = Image.open('examples/ImageRight/04_05_0011_r.png')
image1 = np.array(image1)
image2 = np.array(image2)





'''
plt.imshow(image1, cmap='plasma')
plt.imshow(image2, cmap='plasma')
print(image1)
plt.colorbar()
plt.show()

image1 = np.array(image1)
image2 = np.array(image2)
result1_path = 'output/result1.png'

# image1 = cv2.medianBlur(image1,5)
# image2 = cv2.medianBlur(image2,5)

#cx = 480
#cy = 270
#focal_lingth = 43
disparity = get_disp(image1, image2, result1_path)
#depth = 800000
image_res = Image.open('examples/05_06_0002.png')
image_res = np.array(image_res)
plt.axis('off')
plt.imshow(image_res, cmap='plasma')
plt.colorbar()
plt.show()
#disparity = np.min(disparity)
#baseline = depth * (disparity)/ focal_lingth
#baseline = (800000 * disp_one) / 43
#print(baseline)
'''

result1_path = 'output/result1.png'
image3 = Image.open('examples/ImageLeft/04_05_0011_l.png')
image4 = Image.open('examples/ImageRight/04_05_0011_r.png')
dis_first = Image.open('examples/04_05_0011.png')
image3 = image3.convert('L')
image4 = image4.convert('L')

#dis_first = Image.open('examples/ImageRight/20_29_0005.png')

dis_first = np.array(dis_first)
dis_first = dis_first.astype(float) / 100


image3 = np.array(image3)
image4 = np.array(image4)
image3 = cv2.medianBlur(image3,5)
image4 = cv2.medianBlur(image4,5)




plt.imshow(image3, cmap='plasma')
plt.colorbar()
plt.show()

plt.imshow(image4, cmap='plasma')
plt.colorbar()
plt.show()

# image3 = cv2.medianBlur(image3, 5)
# image4 = cv2.medianBlur(image4,5)
#cx = 480
#cy = 270
#focal_lingth = 43
disparity = get_disp(image3, image4, result1_path)
plt.imshow(dis_first, cmap='plasma')
plt.colorbar()
plt.show()



result1_path = 'output/result1.png'


from PIL import Image

# Open an image file
image5 = Image.open('examples/ImageLeft/Screenshot from 2024-10-09 09-46-04.png')
image6 = Image.open('examples/ImageRight/Screenshot from 2024-10-09 09-46-16.png')
# Define the crop area (left, upper, right, lower)
# Crop 100 pixels from the right side
crop_area = (100, 0, image5.width - 800, image5.height-101)

# Crop the image
image5 = image5.crop(crop_area)

# Show the cropped image

crop_area = (0, 0, image6.width-900, image6.height-100)

# Crop the image
image6 = image6.crop(crop_area)

# Show the cropped image
# Save the cropped image

dis_second =Image.open('examples/04_05_0009.png') 
#depth_three = get_depth(350, 15000 , 10.3,10.3,disp_three)
image5.save('cropped_image5.png')
image6.save('cropped_image6.png')

image5 = np.array(image5)
image6 = np.array(image6)

dis_second = np.array(dis_second)
dis_second = dis_second.astype(float) / 100

#image3 = cv2.medianBlur(image3,7)
#image4 = cv2.medianBlur(image4,7)
plt.imshow(image5, cmap='plasma')
plt.colorbar()
plt.show()

plt.imshow(image6, cmap='plasma')
plt.colorbar()
plt.show()

# image3 = cv2.medianBlur(image3,5)
# image4 = cv2.medianBlur(image4,5)
#cx = 480
#cy = 270
#focal_lingth = 43
disparity = get_disp(image5, image6, result1_path)
#depth_three = get_depth(350, 15000 , 10.3,10.3,disp_three)

plt.imshow(dis_second, cmap='plasma')
plt.colorbar()
plt.show()

In [ ]:
from app import load_model_for_conversion
from app import convert_to_onnx




result1_path = 'output/result1.png'
image5 = Image.open('examples/ImageLeft/20_29_0006.png')
image6 = Image.open('examples/ImageRight/20_29_0006.png')



dis_second =Image.open('examples/20_29_0006.png') 
#depth_three = get_depth(350, 15000 , 10.3,10.3,disp_three)

image5 = np.array(image5)
image6 = np.array(image6)
image5 = cv2.medianBlur(image5,9)
image6 = cv2.medianBlur(image6,9)


dis_second = np.array(dis_second)
dis_second = dis_second.astype(float) / 100

#image3 = cv2.medianBlur(image3,7)
#image4 = cv2.medianBlur(image4,7)
plt.imshow(image5, cmap='plasma')
plt.colorbar()
plt.show()

plt.imshow(image6, cmap='plasma')
plt.colorbar()
plt.show()

# image3 = cv2.medianBlur(image3,5)
# image4 = cv2.medianBlur(image4,5)
#cx = 480
#cy = 270
#focal_lingth = 43
disparity = get_disp(image5, image6, result1_path)
#depth_three = get_depth(350, 15000 , 10.3,10.3,disp_three)

plt.imshow(dis_second, cmap='plasma')
plt.colorbar()
plt.show()

# Load the model

# Path to save the ONNX model
onnx_file_path = "depth_estimation_model.onnx"

# Convert the model to ONNX
convert_to_onnx(onnx_file_path,image5,image6)

In [ ]:

# Load the image
image3 = cv2.imread('examples/ImageLeft/20_29_0005.png')
image4 =  cv2.imread('examples/ImageRight/20_29_0005.png')
disparity = get_disp(image3, image4, result1_path)

# Convert to LAB color space
lab_image = cv2.cvtColor(image3, cv2.COLOR_BGR2LAB)

# Split the LAB image into its channels
l, a, b = cv2.split(lab_image)

# Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to L-channel to enhance contrast
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
l_clahe = clahe.apply(l)

# Merge CLAHE enhanced L-channel with A and B channels
lab_clahe = cv2.merge((l_clahe, a, b))

# Convert LAB back to BGR
image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Shadow detection using thresholding on the L channel (lightness)
# Assuming shadows are in darker regions, we apply a threshold
shadow_mask = cv2.inRange(l, 0, 80)  # Adjust threshold depending on shadow intensity

# Inpainting the shadows to remove them
# You can use different inpainting techniques: 'cv2.INPAINT_TELEA' or 'cv2.INPAINT_NS'
image3 = cv2.inpaint(image_clahe, shadow_mask, 5, cv2.INPAINT_TELEA)


# Convert to LAB color space
lab_image = cv2.cvtColor(image4, cv2.COLOR_BGR2LAB)

# Split the LAB image into its channels
l, a, b = cv2.split(lab_image)

# Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to L-channel to enhance contrast
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
l_clahe = clahe.apply(l)

# Merge CLAHE enhanced L-channel with A and B channels
lab_clahe = cv2.merge((l_clahe, a, b))

# Convert LAB back to BGR
image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Shadow detection using thresholding on the L channel (lightness)
# Assuming shadows are in darker regions, we apply a threshold
shadow_mask = cv2.inRange(l, 0, 80)  # Adjust threshold depending on shadow intensity

# Inpainting the shadows to remove them
# You can use different inpainting techniques: 'cv2.INPAINT_TELEA' or 'cv2.INPAINT_NS'
image4 = cv2.inpaint(image_clahe, shadow_mask, 5, cv2.INPAINT_TELEA)

disparity = get_disp(image3, image4, result1_path)


In [ ]:


#depth = 800000
image_res = Image.open('examples/04_05_0011.png')
image_res = np.array(image_res)
image_res = image_res.astype(float) / 100

plt.axis('off')
plt.imshow(image_res, cmap='plasma')
plt.colorbar()
plt.show()
#disparity = np.min(disparity)
#baseline = depth * (disparity)/ focal_lingth
#baseline = (800000 * disp_one) / 43
#print(baseline)

from PIL import Image
import matplotlib.pyplot as plt
import glob

# Path to the folder containing the .tif files
tif_files = glob.glob('examples/01.tif')

'''
# Loop through each .tif file in the directory
for file in tif_files:
    print(f"Processing {file}")
    tif = Image.open(file)  # Open each TIFF file
    
    # Loop through all the frames in the TIFF file
    for i in range(tif.n_frames):
        tif.seek(i)  # Go to the ith frame
        plt.imshow(tif)

        # Display the current frame
        plt.imshow(tif)
        plt.title(f"{file} - Page {i+1}")  # Set the title to filename and page number
        plt.axis('off')  # Turn off axis
        plt.show()  # Show the image # Show the image
'''

def ras_show_flip(file_path):
    with rasterio.open(file_path) as src:
        
        # Read a small window from the file
        window = src.read(window=((0, 3000), (0, 3000)))

        # Check if the window is 2D (single band) or 3D (multiple bands)
        if window.ndim == 3:
            # First rotate the window by 90 degrees clockwise (for 3D array)
            rotated_window = np.rot90(window, k=3, axes=(1, 2))  # Rotate along height and width axes
            
            # Then flip the rotated window upside-down (axis 1)
            flipped_rotated_window = np.flip(rotated_window, axis=1)  # Flip vertically
            flipped_rotated_window = np.rot90(flipped_rotated_window, k=3, axes=(1, 2))  # Rotate along height and width axes

            # Display the corrected image from band 1
            plt.imshow(flipped_rotated_window[0])
            plt.title('Corrected Image - Band 1')
            plt.axis('off')
            plt.show()

            # Display the corrected image from band 3 (if available)
            if flipped_rotated_window.shape[0] > 2:  # Ensure there are enough bands
                plt.imshow(flipped_rotated_window[2])
                plt.title('Corrected Image - Band 3')
                plt.axis('off')
                plt.show()
        
        elif window.ndim == 2:
            # If the window is 2D (single band), rotate and flip accordingly
            rotated_window = np.rot90(window[0], k=3)  # Rotate 90 degrees clockwise
            flipped_rotated_window = np.flip(rotated_window, axis=1)  # Flip vertically
            rotated_window = np.rot90(window[0], k=3)  # Rotate 90 degrees clockwise

            # Display the corrected image
            plt.imshow(flipped_rotated_window)
            plt.title('Corrected Image')
            plt.axis('off')
            plt.show()

        # Print metadata about the file
        print(f"File Dimensions: {src.width} x {src.height}")
        print(f"Number of Bands: {src.count}")
        print(f"Data Type: {src.dtypes[0]}")
def load_stereo_images(file_path):
    with rasterio.open(file_path) as src:
        
        # Read the two bands (assuming band 1 is left and band 2 is right)
        left_image = src.read(1)  # Band 1 (left image)
        right_image = src.read(5)  # Band 2 (right image)

        # Display left and right images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(left_image, cmap='gray')
        axes[0].set_title('Left Image')
        axes[0].axis('off')

        axes[1].imshow(right_image, cmap='gray')
        axes[1].set_title('Right Image')
        axes[1].axis('off')

        plt.show()

        return left_image, right_image
def ras_show(file_path):
    with rasterio.open(file_path) as src:
        
        # Read a small window to check the data
        window_left = src.read(window=((0, 8000), (0, 8000)))
        rgb_left = window_left[:3].transpose((1, 2, 0))  # Transpose to shape (height, width, 3)

        plt.imshow(rgb_left)
        plt.title('Sample Window of the .tif File')
        plt.show()
        window_right = src.read(window=((0, 3000), (0, 3000)))
        rgb_right = window_right[:3].transpose((1, 2, 0))  # Transpose to shape (height, width, 3)

        plt.imshow(rgb_right)
        plt.title('Sample Window of the .tif File')
        plt.show()
        return rgb_left, rgb_right
#file_path_one = 'IGEV-main/images/05_06_0002.png'
file_path_one = 'examples/10030060.tif'
file_path_two = 'examples/04.tif'
image3, image4 = ras_show(file_path_two)

#disparity = get_disp(image3, image4, result1_path)

#ras_show_flip(file_path_two)
#load_stereo_images(file_path_one)

#ras_show(file_path_two)

In [ ]:
import tifffile as tiff
import numpy as np
from PIL import Image
''''
# Function to extract two near images from a TIFF file
def extract_images_from_tiff(tiff_file, image_index_1, image_index_2):
    # Load the TIFF file
    with tiff.TiffFile(tiff_file) as tif:
        # Extract the desired frames (images)
        image1 = tif.pages[image_index_1].asarray()
        image2 = tif.pages[image_index_2].asarray()
        
        return image1, image2

# Path to your large TIFF file
tiff_file = 'examples/10030060.tif'

# Specify which two images you want to extract from the TIFF (e.g., 0 and 1)
image1, image2 = extract_images_from_tiff(tiff_file, image_index_1=0, image_index_2=1)

# Optionally convert images to PIL format and back to numpy (if needed for your code)
image1 = Image.fromarray(image1)
image2 = Image.fromarray(image2)

image1 = np.array(image1)
image2 = np.array(image2)

# Path to save the disparity result
result1_path = 'path/to/save/disparity.png'

# Calculate disparity using your function
disparity = get_disp(image1, image2, result1_path)


In [ ]:
from PIL import Image

# Open a multi-page TIFF file
tif = Image.open("examples/04.tif")

# Loop through pages/images in the TIFF
for i in range(tif.n_frames):
    tif.seek(i)
    tif.save(f"page_{i}.png")  # Save each image as a separate file

In [ ]:
import rasterio
import matplotlib.pyplot as plt

def load_stereo_images_from_window(file_path, left_window, right_window):
    with rasterio.open(file_path) as src:
        # Read the left image from the specified window
        left_image = src.read(1)
        
        # Rotate the image by 360 degrees
        (h, w) = left_image.shape[:2]
        center = (w // 2, h // 2)
        
        # Compute the rotation matrix
        rotation_matrix = cv2.getRotationMatrix2D(center, 360, 1.0)
        
        # Apply the rotation
        left_image = cv2.warpAffine(left_image, rotation_matrix, (w, h))
        print(left_image.shape)
        # Read band 1 for left image
        #left_image = color_normalization(left_image) 

        # Read the right image from the specified window
        right_image = src.read(1)
        print(right_image.shape)

        # Read band 1 for right image
        #right_image = color_normalization(right_image) 

        # Display left and right images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        axes[0].imshow(left_image)
        axes[0].set_title('Left Image')
        axes[0].axis('off')

        axes[1].imshow(right_image)
        axes[1].set_title('Right Image')
        axes[1].axis('off')

        plt.show()
        return left_image, right_image
# Define your file path
file_path = 'examples/04.tif'

# Load and display stereo images
image1, image2=load_stereo_images_from_window(file_path, left_window, right_window)
from PIL import Image



# Save the image
image1 = image1.astype(np.uint8)  # Convert to uint8 format (required for images)
print(image1)

# Save the image using OpenCV
cv2.imwrite('image1.png', image1)


# Save the image
image2 = image2.astype(np.uint8)  # Convert to uint8 format (required for images)
print(image2)
# Save the image using OpenCV
cv2.imwrite('image2.png', image2)

result1_path = 'output/result1.png'

# image1 = cv2.medianBlur(image1,5)
# image2 = cv2.medianBlur(image2,5)



#cx = 480
#cy = 270
#focal_lingth = 43
disparity = get_disp(image1, image2, result1_path)

In [ ]:
import numpy as np
import cv2
import concurrent.futures
from PIL import Image
model = load_model()

# Load the model only once
def process_stereo_video_for_disparity(video_left_path, video_right_path, output_video_path=None, skip_frames=4):
    # Step 1: Open the left and right video streams
    cap_left = cv2.VideoCapture(video_left_path)
    cap_right = cv2.VideoCapture(video_right_path)
    i = 0

    # Get video properties
    frame_width = int(cap_left.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap_left.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap_left.get(cv2.CAP_PROP_FPS) // skip_frames  # Adjust fps according to skipped frames

    print("Frame width:", frame_width)
    print("Frame height:", frame_height)
    print("FPS:", fps)

    # Video writer to save the output, if specified
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Using XVID codec
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    def process_frame_pair(frame_left, frame_right):
        # Convert frames to PIL format (if needed by your stereo model)
        image_left = Image.fromarray(cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB))
        image_right = Image.fromarray(cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB))
        # Get disparity map for the current frame

        disparity_map = inference(image_left, image_right, model, task='stereo')

        disparity_map = np.array(disparity_map)

        # Normalize disparity map to the range 0-255
        disparity_visual = (disparity_map / np.max(disparity_map) * 255).astype(np.uint8)

        # Apply color map for better visualization
        disparity_color = cv2.applyColorMap(disparity_visual, cv2.COLORMAP_PLASMA)

        # Save the disparity frame to the video file
        if output_video_path:
            out.write(disparity_color)

        return disparity_color

    # Step 2: Process each frame (using multithreading)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while cap_left.isOpened() and cap_right.isOpened():
            i += 1
            
            # Read frames from both the left and right video streams
            ret_left, frame_left = cap_left.read()
            ret_right, frame_right = cap_right.read()  # Now correctly unpacking

            # Check if both frames were successfully read
            if not ret_left or not ret_right:
                print("End of video or error in reading frames.")
                break

            # Skip frames for faster processing
            for _ in range(skip_frames - 1):
                cap_left.grab()
                cap_right.grab()

            # Submit the frame pair to the thread pool for processing
            future = executor.submit(process_frame_pair, frame_left, frame_right)

            # Retrieve processed disparity map from future and show it
            disparity_color = future.result()

            # Optionally: show the disparity map in real-time
            #cv2.imshow('Disparity Map', disparity_color)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Step 3: Release all resources
    cap_left.release()
    cap_right.release()
    print(f"Processed {i} frames.")

    if output_video_path:
        out.release()
    cv2.destroyAllWindows()

# Example usage
video_left_path = 'examples/filename1 (1).avi'
video_right_path = 'examples/filename1 (1).avi'
output_video_path = 'output/depth_video.avi'  # Change extension to .avi

# Process the stereo video and save the disparity output
process_stereo_video_for_disparity(video_left_path, video_right_path, output_video_path=output_video_path, skip_frames=3)

In [ ]:
def read_pfm(file):
    with open(file, 'rb') as f:
        # Read header
        header = f.readline().decode('utf-8').rstrip()
        if header == 'PF':
            color = True
        elif header == 'Pf':
            color = False
        else:
            raise Exception('Not a PFM file.')

        # Read dimensions
        dims = f.readline().decode('utf-8').rstrip()
        width, height = map(int, dims.split())

        # Read scale factor (negative for little endian)
        scale = float(f.readline().decode('utf-8').rstrip())
        endian = '<' if scale < 0 else '>'

        # Read the data
        data = np.fromfile(f, endian + 'f')
        shape = (height, width, 3) if color else (height, width)
        data = np.reshape(data, shape)
        data = np.flipud(data)

    return data


pfm_data_one = read_pfm('examples/DispLeft/dis0010_L.pfm')
pfm_data_two = read_pfm('examples/DispLeft/dis0100_L.pfm')
pfm_data_three = read_pfm('examples/DispLeft/dis0040_L.pfm')
pfm_data_four = read_pfm('examples/DispLeft/dis0120_L.pfm')
depth_one_true = get_depth(350, 15000 , 10.3,10.3, pfm_data_one)  #focal_length, baseline, cx2, cx1, disparity
depth_two_true= get_depth(350, 15000 , 10.3,10.3, pfm_data_two)
depth_three_true = get_depth(350, 15000 , 10.3,10.3, pfm_data_three)  #focal_length, baseline, cx2, cx1, disparity
depth_four_true= get_depth(350, 15000 , 10.3,10.3, pfm_data_four)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import tifffile as tiff


import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt





image3 = tiff.imread('examples/po_97258_pan_0000000.tif')
image4 = tiff.imread('examples/po_97258_pan_0010000.tif')
plt.imshow(image3,cmap='gray')
plt.show()

plt.imshow(image4, cmap='gray')
plt.show()
image3 = cv2.medianBlur(image3, 5)  # 5 is an odd number
image4 = cv2.medianBlur(image4, 5)  # 5 is an odd number
result2_path = 'output/result2.png'
disp_two = get_disp(image3, image4, result2_path)
depth_two= get_depth(350, 15000 , 10.3,10.3, disp_two)


In [ ]:
import matplotlib.pyplot as plt


plt.imshow(depth_one, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
#plt.imshow(depth_one_true, cmap='viridis', interpolation='none')
#plt.colorbar(label='Depth (millimetre)')3
#plt.axis('off')
#plt.show()


plt.imshow(depth_two, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_two_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()



plt.imshow(depth_three, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_three_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()



plt.imshow(depth_four, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_four_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()




In [ ]:
import cv2
import numpy as np

def calculate_baseline(image_left, image_right, camera_matrix):
    # Step 1: Detect ORB features and compute descriptors.
    orb = cv2.ORB_create()
    keypoints_left, descriptors_left = orb.detectAndCompute(image_left, None)
    keypoints_right, descriptors_right = orb.detectAndCompute(image_right, None)

    # Step 2: Match features using BFMatcher (Brute Force Matcher)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors_left, descriptors_right)

    # Sort matches based on distance (best matches first)
    matches = sorted(matches, key=lambda x: x.distance)

    # Step 3: Extract the matched keypoints
    pts_left = np.float32([keypoints_left[m.queryIdx].pt for m in matches])
    pts_right = np.float32([keypoints_right[m.trainIdx].pt for m in matches])

    # Step 4: Compute the essential matrix
    E, _ = cv2.findEssentialMat(pts_left, pts_right, camera_matrix, method=cv2.RANSAC, prob=0.999, threshold=1.0)

    # Step 5: Recover the pose (rotation and translation)
    _, R, t, _ = cv2.recoverPose(E, pts_left, pts_right, camera_matrix)

    # Step 6: Calculate the baseline
    baseline = np.linalg.norm(t)  # The baseline is the magnitude of the translation vector

    return baseline

# Example usage:
image_left = cv2.imread('examples/ImageLeft/0100_L.png', 0)
image_right = cv2.imread('examples/ImageRight/0100_R.png', 0)


# Replace with your actual camera matrix
camera_matrix = np.array([[993, 0, 480],
                          [0, 993, 270],
                          [0, 0, 1]])

baseline = calculate_baseline(image_left, image_right, camera_matrix)
print(f'Baseline (in meters): {baseline}')

In [ ]:
# Test

batch_size = 1
height = 640
width = 960
input_channels = 3
dummy_input1 = torch.randn(batch_size, input_channels, height, width)  # Static input sizes
print(dummy_input1.shape)
dummy_input2 = torch.randn(batch_size, input_channels, height, width)
# Move inputs to CUDA if available
if torch.cuda.is_available():
    dummy_input1 = dummy_input1.cuda()
    dummy_input2 = dummy_input2.cuda()

concat_tensor = torch.cat((dummy_input1, dummy_input2), dim=0)

concat_tensor.shape
